In [1]:
import pandas as pd


from google.colab import files
uploaded=files.upload()

Saving anime.csv to anime.csv


In [ ]:
anime_data = pd.read_csv(r"Recommendation_System.csv")
anime_data


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


# Data Exploration and Preprocessing



In [ ]:

missing_values = anime_data.isnull().sum()
data_types = anime_data.dtypes
missing_values, data_types

(anime_id      0
 name          0
 genre        62
 type         25
 episodes      0
 rating      230
 members       0
 dtype: int64,
 anime_id      int64
 name         object
 genre        object
 type         object
 episodes     object
 rating      float64
 members       int64
 dtype: object)

In [ ]:
anime_data['genre'].fillna('Unknown', inplace=True)
anime_data['type'].fillna('Unknown', inplace=True)
anime_data['rating'].fillna(anime_data['rating'].mean(), inplace=True)
anime_data['episodes'] = pd.to_numeric(anime_data['episodes'], errors='coerce')
anime_data['episodes'].fillna(anime_data['episodes'].mean(), inplace=True)
anime_data.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

# Feature Extraction

In [ ]:


from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
genres_encoded = anime_data['genre'].str.get_dummies(sep=', ')
scaler = MinMaxScaler()
anime_data[['rating', 'members']] = scaler.fit_transform(anime_data[['rating', 'members']])
features = pd.concat([anime_data[['rating', 'members']], genres_encoded], axis=1)
features

,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri
0,0.924370,0.197872,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,0.911164,0.782770,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0.909964,0.112689,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.900360,0.664325,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0.899160,0.149186,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0.297719,0.000203,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,0.313325,0.000176,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,0.385354,0.000211,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,0.397359,0.000168,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

anime_data['genre'].fillna('Unknown', inplace=True)
anime_data['type'].fillna('Unknown', inplace=True)
anime_data['rating'].fillna(anime_data['rating'].mean(), inplace=True)
anime_data['episodes'] = pd.to_numeric(anime_data['episodes'], errors='coerce')
anime_data['episodes'].fillna(anime_data['episodes'].mean(), inplace=True)
genres_encoded = anime_data['genre'].str.get_dummies(sep=', ')


scaler = MinMaxScaler()
anime_data[['rating', 'members']] = scaler.fit_transform(anime_data[['rating', 'members']])
features = pd.concat([anime_data[['rating', 'members']], genres_encoded], axis=1)


def recommend_anime(target_anime, features, anime_data, threshold=0.5):
    if target_anime not in anime_data['name'].values:
        return []
    cosine_sim = cosine_similarity(features)
    target_idx = anime_data[anime_data['name'] == target_anime].index[0]
    sim_scores = list(enumerate(cosine_sim[target_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_indices = [i[0] for i in sim_scores if i[1] > threshold and i[0] != target_idx]
    return anime_data.iloc[sim_indices]['name'].tolist()

target_anime = "Kimi no Na wa."
recommended_anime = recommend_anime(target_anime, features, anime_data)
print("Recommended anime for {}: {}".format(target_anime, recommended_anime))

Recommended anime for Kimi no Na wa.: ['Wind: A Breath of Heart OVA', 'Wind: A Breath of Heart (TV)', 'Aura: Maryuuin Kouga Saigo no Tatakai', 'Clannad: After Story - Mou Hitotsu no Sekai, Kyou-hen', 'Kokoro ga Sakebitagatterunda.', 'Angel Beats!: Another Epilogue', 'True Tears', 'Myself; Yourself', 'Kimikiss Pure Rouge', 'Koi to Senkyo to Chocolate', 'Harmonie', 'Air Movie', '&quot;Bungaku Shoujo&quot; Memoire', 'Koi to Senkyo to Chocolate Special', 'Otome wa Boku ni Koishiteru: Futari no Elder', 'Myself ; Yourself Specials', 'Koi to Senkyo to Chocolate: Ikenai Hazuki-sensei', 'Touka Gettan', 'Venus Versus Virus', 'Mizuiro (2003)', 'To Heart 2 Special', 'School Days ONA', 'Tokimeki Memorial: Forever With You', 'Little Busters!: Refrain', 'Kokoro Connect', 'Kokoro Connect: Michi Random', 'Shakugan no Shana', 'Shakugan no Shana II (Second)', 'Clannad', 'Little Busters!: EX', 'Shakugan no Shana S', 'Hotarubi no Mori e', 'Yahari Ore no Seishun Love Comedy wa Machigatteiru. Zoku', 'Kanon (

# Evaluation

In [ ]:

def evaluate_recommendations(anime_data, features, threshold=0.5):
    train_data, test_data = train_test_split(anime_data, test_size=0.2, random_state=42)
    train_features = features.loc[train_data.index]
    test_features = features.loc[test_data.index]
    y_true = []
    y_pred = []
    for idx, row in test_data.iterrows():
        target_anime = row['name']
        true_anime = test_data[test_data['genre'] == row['genre']]['name'].tolist()
        recommended_anime = recommend_anime(target_anime, train_features, train_data, threshold)
        y_true.extend([1 if anime in true_anime else 0 for anime in test_data['name']])
        y_pred.extend([1 if anime in recommended_anime else 0 for anime in test_data['name']])
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    f1 = f1_score(y_true, y_pred, average='micro')

    return precision, recall, f1

precision, recall, f1 = evaluate_recommendations(anime_data, features)
print("Precision: {:.2f}, Recall: {:.2f}, F1-score: {:.2f}".format(precision, recall, f1))

Precision: 0.99, Recall: 0.99, F1-score: 0.99
